File to compute the baselines with skitlearn

In [1]:
# from master import MASTERModel
import pickle
import numpy as np
import time
import os

# Please install qlib first before load the data.

universe = 'csi300' # ['csi300','csi800']
prefix = 'opensource' # ['original','opensource'], which training data are you using
train_data_dir = 'data'
with open(os.path.join(train_data_dir, prefix, f"{universe}_dl_train.pkl"), 'rb') as f:
    dl_train = pickle.load(f)

predict_data_dir = os.path.join('data', prefix)
with open(os.path.join(predict_data_dir, f"{universe}_dl_valid.pkl"), 'rb') as f:
    dl_valid = pickle.load(f)
with open(os.path.join(predict_data_dir, f"{universe}_dl_test.pkl"), 'rb') as f:
    dl_test = pickle.load(f)


print(len(dl_train), len(dl_valid), len(dl_test))

856246 20039 185700


In [2]:
# grab the very first sample
sample = dl_train[0]

# this will print something like (step_len, num_features)
print("Sample shape:", sample.shape)

# so the number of features is the second entry:
print("Number of features:", sample.shape[1])


Sample shape: (8, 222)
Number of features: 222


In [3]:
first_element = dl_train[20]
print(first_element)

[[ 0.34614566  0.5675101   0.23646091 ... -0.861093   -0.09485655
  -0.03220981]
 [ 0.34614566  0.5675101   0.23646091 ... -0.861093   -0.09485655
  -0.03220981]
 [ 0.34614566  0.5675101   0.23646091 ... -0.861093   -0.09485655
  -0.03220981]
 ...
 [ 0.34614566  0.5675101   0.23646091 ... -0.861093   -0.09485655
  -0.03220981]
 [ 0.34614566  0.5675101   0.23646091 ... -0.861093   -0.09485655
  -0.03220981]
 [ 0.34614566  0.5675101   0.23646091 ... -0.861093   -0.09485655
  -0.03220981]]


In [4]:
sample = dl_train[0]
print(type(sample))
print(len(sample))
print(sample)

<class 'numpy.ndarray'>
8
[[ 0.5669539   1.7702353   0.25422162 ... -0.861093   -0.09485655
   0.11779058]
 [ 0.5669539   1.7702353   0.25422162 ... -0.861093   -0.09485655
   0.11779058]
 [ 0.5669539   1.7702353   0.25422162 ... -0.861093   -0.09485655
   0.11779058]
 ...
 [ 0.5669539   1.7702353   0.25422162 ... -0.861093   -0.09485655
   0.11779058]
 [ 0.5669539   1.7702353   0.25422162 ... -0.861093   -0.09485655
   0.11779058]
 [ 0.5669539   1.7702353   0.25422162 ... -0.861093   -0.09485655
   0.11779058]]


In [5]:
features = sample[:, :-1]
labels = sample[:, -1]
print("Features shape:", features.shape)
print("Labels shape:", labels.shape)
print("First row of features:", features[0])
print("First label:", labels[0])

Features shape: (8, 221)
Labels shape: (8,)
First row of features: [ 5.6695390e-01  1.7702353e+00  2.5422162e-01  3.0000000e+00
  9.5317376e-01  2.6684573e+00  6.4764029e-01  2.1456070e-01
  9.5380224e-02 -5.6113130e-01  1.2409714e+00 -1.6652515e+00
  8.9781553e-01 -1.3754560e+00 -1.9777788e+00  1.6816553e-04
 -4.4862938e-01  1.0131649e-01 -1.6153438e+00 -1.9349709e+00
 -9.2531496e-01 -8.9392287e-01 -2.4580985e-01  9.1333961e-01
  1.1013550e+00  8.5150141e-01  2.7503508e-01 -6.1919149e-02
  1.6474377e+00  1.8437111e+00 -5.7491177e-01  1.5725052e-01
 -5.3635913e-01  9.3971366e-01  1.1357726e+00 -8.3118469e-01
 -1.0965987e+00 -3.7234446e-01  6.1236262e-02  2.4267231e-01
  2.4795330e+00  1.1651956e+00  1.1864377e+00 -1.7535737e-01
 -4.8689702e-01 -3.8772559e-01 -5.4134125e-01 -1.8199921e-02
 -1.5110452e+00 -2.4794333e+00 -1.4300404e+00 -9.7289109e-01
 -3.7831423e-01 -9.7764313e-01 -1.4713941e+00 -4.4978639e-01
 -5.7074600e-01 -2.2540481e-01 -2.9431176e+00 -2.4316034e+00
 -1.3298811e+00 -1

In [6]:
from master import MASTERModel
import pickle
import numpy as np
import time
import os

d_feat = 158
d_model = 256
t_nhead = 4
s_nhead = 2
dropout = 0.5
gate_input_start_index = 158
gate_input_end_index = 221

if universe == 'csi300':
    beta = 5
elif universe == 'csi800':
    beta = 2

n_epoch = 1
lr = 1e-5
GPU = 0
train_stop_loss_thred = 0.95


ic = []
icir = []
ric = []
ricir = []

# Training
######################################################################################
for seed in [0, 1]:
    model = MASTERModel(
        d_feat = d_feat, d_model = d_model, t_nhead = t_nhead, s_nhead = s_nhead, T_dropout_rate=dropout, S_dropout_rate=dropout,
        beta=beta, gate_input_end_index=gate_input_end_index, gate_input_start_index=gate_input_start_index,
        n_epochs=n_epoch, lr = lr, GPU = GPU, seed = seed, train_stop_loss_thred = train_stop_loss_thred,
        save_path='model', save_prefix=f'{universe}_{prefix}'
    )

    start = time.time()
    # Train
    model.fit(dl_train, dl_valid)

    print("Model Trained.")

    # Test
    predictions, metrics = model.predict(dl_test)
    
    running_time = time.time()-start
    
    print('Seed: {:d} time cost : {:.2f} sec'.format(seed, running_time))
    print(metrics)

    ic.append(metrics['IC'])
    icir.append(metrics['ICIR'])
    ric.append(metrics['RIC'])
    ricir.append(metrics['RICIR'])

/home/gabrielecarrino/Master-ANN/MASTER_original/base_model.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.daily_count = pd.Series(index=self.data_source.get_index()).groupby("datetime").size().values


Feature shape: torch.Size([277, 8, 221])
Label shape: torch.Size([277])
Pred shape: torch.Size([277])
Loss shape: torch.Size([])
Feature: tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  6.9120e-03,
           5.4797e-01,  6.5908e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  6.9120e-03,
           5.4797e-01,  6.5908e-01],
         [ 7.8048e-01,  7.2385e-01,  4.9920e-01,  ..., -3.1058e-02,
           7.3069e-01,  8.6529e-01],
         ...,
         [ 1.4830e+00,  1.8501e-01,  1.2154e+00,  ...,  4.5389e-04,
           2.2899e+00,  2.6647e+00],
         [ 1.1323e+00,  4.2756e-01,  8.2368e-01,  ...,  7.1963e-02,
           4.0629e-01,  7.9544e-01],
         [ 1.0089e-01,  1.4637e-02,  9.0763e-02,  ...,  6.9110e-02,
           2.3760e-01,  6.3281e-01]],

        [[-1.0533e+00, -2.9021e-01, -1.1482e+00,  ...,  8.3602e-03,
           2.3011e-01,  3.6598e-01],
         [-1.8678e-01, -4.0873e-01, -2.2187e-01,  ...,  6.9120e-03,
           5.4797e-01,  6.5908e-01],
      

NameError: name 'exit' is not defined